In [12]:
import cv2 as cv
import numpy as np
import string
import os
from copy import deepcopy
from cairosvg import svg2png
from wand.image import Image


In [13]:
captcha_path = './../Generators/captchas/svg_captcha'
images_path = '/images'
labels_path = '/labels'
letters_path = '/letters'

images = []
labels = []
letters = []

def get_data(directory):
    result = []
    for filename in os.listdir(directory):
        f = os.path.join(directory,filename)
        result.append(f)
    result.sort()
    return result

def show_image(title,image):
    cv.imshow(title,image)
    cv.waitKey(0)
    cv.destroyAllWindows()

def show_image_negative(title,image):
    image_n=cv.bitwise_not(image)
    cv.imshow(title,image_n)
    cv.waitKey(0)
    cv.destroyAllWindows()

image_files = get_data(captcha_path+images_path)
label_files = get_data(captcha_path+labels_path)
letter_files = get_data(captcha_path+letters_path)

In [14]:
import os

formated_svg_path = './svg_formated'
formated_let_path = './let_formated'
cropped_let_path = './let_cropped'

def make_dir(dir_path):
    # check whether directory already exists
    if not os.path.exists(dir_path):
      os.mkdir(dir_path)
      print("Folder %s created!" % dir_path)
    else:
      print("Folder %s already exists" % dir_path)
    
make_dir(formated_svg_path)
make_dir(formated_let_path)
make_dir(cropped_let_path)

for idx in range(len(image_files)):
    fn = image_files[idx]
    ny = Image(filename = fn)
    fn = fn[:-4]
    ny_convert = ny.convert('png')
    ny_convert.save(filename = f'./{formated_svg_path}/{fn.rsplit("/", 1)[-1]}.png')

for idx in range(len(letter_files)):
    fn = letter_files[idx]
    ny = Image(filename = fn)
    fn = fn[:-4]
    ny_convert = ny.convert('png')
    ny_convert.save(filename = f'./{formated_let_path}/{fn.rsplit("/", 1)[-1]}.png')

Folder ./svg_formated already exists
Folder ./let_formated already exists
Folder ./let_cropped already exists


In [ ]:
captcha_path = './svg_formated_test'
captcha_files = get_data(captcha_path)

captcha_array = []
captcha_labels = []

for captcha_file in captcha_files:
    image = cv.imread(captcha_file)
    captcha_array.append(image)
    captcha_labels.append(captcha_file[captcha_file.rfind('/')+1:].split('.')[0])

In [ ]:
np.save("svg_formated_test_array",captcha_array)
np.save("svg_formated_test_labels",captcha_labels)

In [ ]:
decoded_samir_path = './samir_decoded/'
make_dir(decoded_samir_path)


captcha_path = './../Generators/captchas/samir_captcha/images'
captcha_files = get_data(captcha_path)

captcha_array = []
captcha_labels = []


    
import base64
from PIL import Image
from io import BytesIO

def base64_to_image(base64_string):
    
    def pil_to_cv(image):
        # Convert PIL Image to RGB mode if it is not
        if image.mode != 'RGB':
            image = image.convert('RGB')

        # Convert PIL Image to OpenCV format
        cv_image = cv.cvtColor(np.array(image), cv.COLOR_RGB2BGR)

        return cv_image

    # Remove the data URI scheme if it exists
    if base64_string.startswith('data:image'):
        base64_string = base64_string.split(',')[1]

    # Decode the Base64 string
    image_data = base64.b64decode(base64_string)

    # Create a PIL Image object from the decoded image data
    image = Image.open(BytesIO(image_data))

    return pil_to_cv(image)

for captcha_file in captcha_files:
    file = open(captcha_file)
    encoded_data = file.read()
    file.close()
    
    base64_string = encoded_data
    image = base64_to_image(base64_string)
    
    cv.imwrite(decoded_samir_path + captcha_file[captcha_file.rfind('/')+1:].split('.')[0]+'.png', image)
